In [1]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [2]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [3]:
# To compare different trajectories
import numpy as np
import shapely
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
from shapely import geometry
from scipy.spatial.distance import directed_hausdorff

In [4]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sdunp = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la_pilot")

About to retrieve messages using {'user': 'shankari@eecs.berkeley.edu', 'key_list': ['config/evaluation_spec'], 'start_time': 0, 'end_time': 1564298416}
response = <Response [200]>
Found 10 entries
After iterating over 10 entries, entry found
Found spec = Pilot car and bike trip in the South Bay
Evaluation ran from 2019-07-19T17:00:00-07:00 -> 2019-07-30T17:00:00-07:00


In [5]:
import importlib
importlib.reload(ezev)

<module 'emeval.viz.eval_view' from '/Users/harivenugopalan/e-mission-eval-public-data/emeval/viz/eval_view.py'>

In [ ]:
pvunp = eipv.PhoneView(sdunp)

-------------------- About to read transitions from server --------------------
Reading data for android phones
Loading transitions for phone ucb-sdb-android-1
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563580800, 'end_time': 1564531200}
response = <Response [200]>
Found 30 entries
Loading transitions for phone ucb-sdb-android-2
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563580800, 'end_time': 1564531200}
response = <Response [200]>
Found 30 entries
Loading transitions for phone ucb-sdb-android-3
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['manual/evaluation_transition'], 'start_time': 1563580800, 'end_time': 1564531200}
response = <Response [200]>
Found 30 entries
Loading transitions for phone ucb-sdb-android-4
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['manual/

response = <Response [200]>
Found 20 entries
About to retrieve messages using {'user': 'ucb-sdb-android-3', 'key_list': ['background/battery'], 'start_time': 1563747211.442, 'end_time': 1563757478.702}
response = <Response [200]>
Found 20 entries
About to retrieve messages using {'user': 'ucb-sdb-android-4', 'key_list': ['background/battery'], 'start_time': 1563747181.383, 'end_time': 1563757442.116}
response = <Response [200]>
Found 9 entries
Processing data for ios phones
About to retrieve messages using {'user': 'ucb-sdb-ios-1', 'key_list': ['background/battery'], 'start_time': 1563747320.468013, 'end_time': 1563757520.8889952}
response = <Response [200]>
Found 23 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-2', 'key_list': ['background/battery'], 'start_time': 1563747247.285541, 'end_time': 1563757355.35479}
response = <Response [200]>
Found 39 entries
About to retrieve messages using {'user': 'ucb-sdb-ios-3', 'key_list': ['background/battery'], 'start_time': 1563

response = <Response [200]>
Found 671 entries
Retrieved 671 entries with timestamps [1563747301.08, 1563747301.109, 1563747314.355, 1563747329.901, 1563747329.929, 1563747345.116, 1563747360.415, 1563747375.537, 1563747390.73, 1563747405.976]...
About to retrieve data for ucb-sdb-android-1 from 1563757428.515 -> 1563757431.506
About to retrieve messages using {'user': 'ucb-sdb-android-1', 'key_list': ['background/motion_activity'], 'start_time': 1563757428.515, 'end_time': 1563757431.506}
response = <Response [200]>
Found 1 entries
Retrieved 1 entries with timestamps [1563757428.515]...
motion activity has not been processed, copying write_ts -> ts
About to retrieve data for ucb-sdb-android-2 from 1563747257.497 -> 1563757403.282
About to retrieve messages using {'user': 'ucb-sdb-android-2', 'key_list': ['background/motion_activity'], 'start_time': 1563747257.497, 'end_time': 1563757403.282}
response = <Response [200]>
Found 402 entries
Retrieved 402 entries with timestamps [1563747274

In [ ]:
polygons = []
onward_trip = pvunp.spec_details.curr_spec['evaluation_trips'][0]

for leg in onward_trip['legs']:
    if 'loc' in leg and leg['loc']['geometry']['type'] == 'Polygon':
        polygons.append(geometry.Polygon(leg['loc']['geometry']['coordinates'][0]))
    if 'end_loc' in leg and leg['end_loc']['geometry']['type'] == 'Polygon':
        polygons.append(geometry.Polygon(leg['end_loc']['geometry']['coordinates'][0]))
    if 'start_loc' in leg and leg['start_loc']['geometry']['type'] == 'Polygon':
        polygons.append(geometry.Polygon(leg['start_loc']['geometry']['coordinates'][0]))

In [ ]:
def clean(arr):
    """
    Latitude and Longitude defined as np array with shape (n,)
    Reshape to (n,)
    """
    return arr.values.reshape((arr.shape[0], 1))

def is_point_in_polygon(point):
    """
    Utility function to check if given Shapely point
    is contained within a Shapely polygon
    """
    for polygon in polygons:
        if polygon.contains(point):
            return True
    return False

def get_trajectory(df, include_timestamps=True):
    """ 
    Returns np array combining latitudes, longitudes
    and timestamps (if asked for). Only returns those
    points that are not contained within Parking Polygons.
    """
    latitudes = clean(df['latitude'])
    longitudes = clean(df['longitude'])
    if include_timestamps:
        timestamps = clean(df['ts'])
    
    coordinates = np.concatenate((latitudes, longitudes), axis=1).tolist()
    
    data_points = []
    
    for (i, coordinate) in enumerate(coordinates):
        point = geometry.Point(coordinate[1], coordinate[0])
        
        if not is_point_in_polygon(point):
            if include_timestamps:
                coordinate.append(timestamps[i][0])
            data_points.append(coordinate)
    
    return np.asanyarray(data_points)

def get_gt_points(pvunp):
    
    # Get lat-long corrdinates in ground truth
    gt_points = []
    onward_trip = pvunp.spec_details.curr_spec['evaluation_trips'][0]
    
    for leg in onward_trip['legs']:
        if 'route_coords' in leg:
            gt_points += leg['route_coords']['geometry']['coordinates']
    return np.asarray(gt_points)

def get_measured_points(df):
    # get lat-long coordinates recorded from phone
    return get_trajectory(df, include_timestamps=False)

def compute_hausdorff_similarity(base_df, test_df):
    # Compute Hausfdorff distance between lat-long-timestamp
    # triplets of different measurements.
    
    base_trajectory = get_trajectory(base_df)
    test_trajectory = get_trajectory(test_df)
    return directed_hausdorff(base_trajectory, test_trajectory)[0]

def plot_splines(gt_spline, measured_spline):
    plt.plot(gt_points[:,1], gt_spline(gt_points[:,1]), 'g')
    plt.plot(measured_points[:,0], measured_spline(measured_points[:,0]), 'r')
    plt.legend(['GT', 'Measured'], loc='best')
    plt.show()
    
def get_error(gt_spline, measured_spline):
    # Error computed by taking absolute difference of integrals of
    # ground truth and measured data
    return abs(abs(measured_spline.integral(np.amin(gt_points[:,1]), np.amax(gt_points[:,1]))) - abs(gt_spline.integral(np.amin(measured_points[:,0]), np.amax(measured_points[:,0]))))

In [ ]:
measured_df = pvunp.phone_view_map['ios']['ucb-sdb-ios-2']['evaluation_ranges'][0]['location_df']

gt_points = get_gt_points(pvunp)
measured_points = get_measured_points(measured_df)
gt_spline = UnivariateSpline(gt_points[:,1], gt_points[:,0])
measured_spline = UnivariateSpline(measured_points[:,0], measured_points[:,1])

In [ ]:
plot_splines(gt_spline, measured_spline)

In [ ]:
get_error(gt_spline, measured_spline)